In [1]:
import findspark
findspark.init("/home/jean/spark-2.4.4-bin-hadoop2.7")
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName('sleep').getOrCreate()

In [11]:
from pyspark.sql.types import DateType
from pyspark.sql.functions import (hour, minute, unix_timestamp,
                                   from_unixtime, hour,datediff,
                                   mean, month)
from pyspark.sql.types import (StructField, StringType,
                               IntegerType, StructType, TimestampType)

In [3]:
df = spark.read.csv('SLEEP/SLEEP_1568018402172.csv', inferSchema=True, header=True)

In [4]:
df.printSchema()

root
 |-- date: timestamp (nullable = true)
 |-- lastSyncTime: integer (nullable = true)
 |-- deepSleepTime: integer (nullable = true)
 |-- shallowSleepTime: integer (nullable = true)
 |-- wakeTime: integer (nullable = true)
 |-- start: integer (nullable = true)
 |-- stop: integer (nullable = true)



In [5]:
df.head()

Row(date=datetime.datetime(2019, 7, 12, 0, 0), lastSyncTime=1563000976, deepSleepTime=98, shallowSleepTime=304, wakeTime=2, start=1562915520, stop=1562939760)

In [6]:
df = df.withColumn("dateStart", from_unixtime(df["start"]))
df = df.withColumn("dateStop", from_unixtime(df["stop"]))
df = df.withColumn("diffHour", (datediff(df["dateStop"], df["dateStart"])* 24) +
                   hour(df["dateStop"]) - hour(df["dateStart"]))

In [7]:
df.printSchema()

root
 |-- date: timestamp (nullable = true)
 |-- lastSyncTime: integer (nullable = true)
 |-- deepSleepTime: integer (nullable = true)
 |-- shallowSleepTime: integer (nullable = true)
 |-- wakeTime: integer (nullable = true)
 |-- start: integer (nullable = true)
 |-- stop: integer (nullable = true)
 |-- dateStart: string (nullable = true)
 |-- dateStop: string (nullable = true)
 |-- diffHour: integer (nullable = true)



In [8]:
df.select(mean(df["diffHour"]).alias("average sleep")).show()

+-----------------+
|    average sleep|
+-----------------+
|7.016949152542373|
+-----------------+



In [12]:
df.groupBy(month(df["date"])).mean().select(["month(date)","avg(diffHour)"]);.

+-----------+-------------------+------------------+---------------------+-----------------+-------------------+-------------------+-----------------+
|month(date)|  avg(lastSyncTime)|avg(deepSleepTime)|avg(shallowSleepTime)|    avg(wakeTime)|         avg(start)|          avg(stop)|    avg(diffHour)|
+-----------+-------------------+------------------+---------------------+-----------------+-------------------+-------------------+-----------------+
|          9|   1.567720877625E9|             76.25|              330.125|              1.5|     1.5676101525E9|      1.567634625E9|            6.875|
|          8|1.566039529064516E9|              85.0|                327.0|4.064516129032258|1.565926691612903E9|1.565951655483871E9|6.903225806451613|
|          7|     1.5638266179E9|              89.3|               345.15|              2.6|       1.56372423E9|      1.563750453E9|             7.25|
+-----------+-------------------+------------------+---------------------+-----------------+--